# Import Dependencies and Dataset

In [6]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras

df = pd.read_csv('output.csv')
df.head()

,box_width,box_height,box_diagonal,class_height,class_width,class_depths,distance
0,0.342187,0.716667,0.509750,4.928,3.084,2.241,11
1,0.340625,0.714583,0.508000,4.928,3.084,2.241,11
2,0.326562,0.693750,0.491500,4.928,3.084,2.241,11
3,0.250000,0.418750,0.321125,3.079,2.619,2.619,11
4,0.268750,0.435417,0.338375,3.079,2.619,2.619,11


In [7]:
# Convert to float32
for cols in df.columns:
    df[cols] = df[cols].astype(float)

In [8]:
df.head()

,box_width,box_height,box_diagonal,class_height,class_width,class_depths,distance
0,0.342187,0.716667,0.509750,4.928,3.084,2.241,11.0
1,0.340625,0.714583,0.508000,4.928,3.084,2.241,11.0
2,0.326562,0.693750,0.491500,4.928,3.084,2.241,11.0
3,0.250000,0.418750,0.321125,3.079,2.619,2.619,11.0
4,0.268750,0.435417,0.338375,3.079,2.619,2.619,11.0


# Building Regression MLP

In [16]:
# Split data
X, y = df.iloc[:, :-1], df.iloc[:, -1]

In [41]:
# Create train and test set (last 10 images were already dedicated for test set beforehand)
X_train, X_test = X.iloc[:-10, ].to_numpy(), X.iloc[-10:, ].to_numpy()
y_train, y_test = y.iloc[:-10, ].to_numpy(), y.iloc[-10:, ].to_numpy()

In [48]:
# Scale the data as this is a regression problem
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [40]:
# model architecture
model = keras.models.Sequential([
    keras.layers.Dense(100, activation='relu', input_shape=(6,)),
    keras.layers.Dropout(0.4),
    
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.4),
    
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dropout(0.4),
    
    keras.layers.Dense(1)
])

model.compile(loss='mean_squared_error', optimizer='sgd')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 100)               700       
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_10 (Dense)            (None, 100)               10100     
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_11 (Dense)            (None, 100)               10100     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_12 (Dense)            (None, 1)                

In [52]:
history = model.fit(X_train, y_train, epochs=30)

Epoch 1/30
1/1 [==============================] - 0s 4ms/step - loss: 9.6134
Epoch 2/30
1/1 [==============================] - 0s 3ms/step - loss: 14.0972
Epoch 3/30
1/1 [==============================] - 0s 4ms/step - loss: 6.7523
Epoch 4/30
1/1 [==============================] - 0s 5ms/step - loss: 4.1441
Epoch 5/30
1/1 [==============================] - 0s 5ms/step - loss: 6.2415
Epoch 6/30
1/1 [==============================] - 0s 6ms/step - loss: 4.1858
Epoch 7/30
1/1 [==============================] - 0s 4ms/step - loss: 5.5455
Epoch 8/30
1/1 [==============================] - 0s 6ms/step - loss: 13.1488
Epoch 9/30
1/1 [==============================] - 0s 4ms/step - loss: 7.7648
Epoch 10/30
1/1 [==============================] - 0s 6ms/step - loss: 5.0274
Epoch 11/30
1/1 [==============================] - 0s 5ms/step - loss: 7.6860
Epoch 12/30
1/1 [==============================] - 0s 6ms/step - loss: 10.1630
Epoch 13/30
1/1 [==============================] - 0s 5ms/step - loss:

# Evaluate on test set

In [56]:
mse_test = model.evaluate(X_test, y_test)
X_new = X_test[3:]
y_pred = model.predict(X_new)

1/1 [==============================] - 0s 14ms/step


#### Visualize the values from test set and predicted set

In [57]:
y_test[3:]

array([13., 15., 15., 17., 17., 20., 20.])

In [58]:
y_pred

array([[12.905873],
       [14.787937],
       [14.560485],
       [15.935598],
       [17.139637],
       [17.27879 ],
       [18.465734]], dtype=float32)